pip install docx2txt


pip install wikipedia -q

In [1]:
from IPython.display import display, Markdown

In [2]:
import getpass
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI


In [3]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv('./.env')

True

In [4]:
os.environ.get('PINECONE_KEY')

'f236a46e-e0e7-4399-a0b2-cb4b8e44bfd8'

def load_document(file): \
    from langchain.document_loaders import PyPDFLoader\
    print(f'Loading {file}')\
    loader = PyPDFLoader(file)\
    data = loader.load()\
    return data

In [ ]:
#print(data[1].page_content)\
#print(data[0].metadata) #page numbering starts with 0\
#print(f'you have {len(data)} pages in your data')\
#print(f'There are {len(data[20].page_content)} characters in your page')

In [6]:
def load_document(file):
    name, extension = os.path.splitext(file)
    
    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
        
    else:
        print("Unsupported Format")
        return(None)
    
    data = loader.load()
    return data

#### Running the Code

In [ ]:
data1 = load_document("D:/Books/Final Year 1st_Sem/Industrial Training/HPPWD.docx")

In [ ]:
print(data1[0].page_content)

### Public and Private Service Loaders

In [9]:
data = load_document("D:/LangChain courses/LLM Project/[CourseClub.Me] ZerotoMastery - Build an LLM-powered QA App using LangChain, OpenAI and Python/code/Notebooks/files/us_constitution.pdf")

Loading D:/LangChain courses/LLM Project/[CourseClub.Me] ZerotoMastery - Build an LLM-powered QA App using LangChain, OpenAI and Python/code/Notebooks/files/us_constitution.pdf


In [69]:
type(data)

list

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 256,
    chunk_overlap=0,
    length_function=len
)


#### .create_documents()
 is used when starting with raw text and creating Document objects from it.
#### .split_documents()
is used when you already have Document objects and want to split their content into smaller chunks.

chunks = text_splitter.create_documents([str(data)])\
print(chunks)

In [10]:
chunks = text_splitter.split_documents(data)


In [11]:
print(len(chunks))

190


In [12]:
print(chunks[10].page_content)

Representatives
shall
chuse
their
Speaker
and
other
Of ficers;and
shall
have
the
sole
Power
of
Impeachment.
Section
3:
The
Senate
The
Senate
of
the
United
States
shall
be
composed
of
two
Senators
from
each
State,
chosen
by
the
Legislature
thereof,
for
six


In [ ]:
import tiktoken

In [ ]:
def print_embedding_cost(texts):
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 *0.0004:.6f}')

print_embedding_cost(chunks)

#### Embedding and Uploading

In [13]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [14]:
import pinecone
from langchain.vectorstores import Pinecone

In [40]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.environ.get('PINECONE_KEY'))

In [42]:
def insert_embeddings(index_name):
    if index_name in pc.list_indexes():
        print(f'Index {index_name} already exists. loading embeddings.', end = '')
        vector_store = pc.from_existing_index(index_name, embeddings)
        print('done')
    else:
        print(f'Creating index {index_name} and embeddings.',end = '')
        vector_store =  pc.create_index(index_name, dimension=768,metric='cosine',
        spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"))
        index = pc.Index(index_name)
        embeddings = [gemini_embeddings.embed_query(doc.page_content) for doc in chunks]
        vectors = []
        for i, embedding in enumerate(embeddings):
            chunk_id = f"chunk_{i}"
            metadata = {"text": chunks[i].page_content}
            vectors.append((chunk_id, embedding, metadata))
            index.upsert(vectors)
        

In [43]:
def delete_pinecone_index(index_name = 'all'):
    if index_name == 'all':
        indexes = pc.list_indexes().names()
        print('Deleting all indexes..')
        for index in indexes:
            pc.delete_index(index)
            print('done')
    else:
        print(f' Deleting index {index_name} ... ', end = '')
        pc.delete_index(index_name)
        print('done')

In [19]:
delete_pinecone_index()


Deleting all indexes..
done


In [20]:
index_name = 'askadocument'
vector_store = insert_embeddings(index_name)

Creating index askadocument and embeddings.

#### Asking and Getting Answers

In [44]:
index_name = 'askadocument'

In [45]:
gemini_api_key = os.environ.get('GOOGLE_API_KEY')

In [46]:
genai.configure(api_key=gemini_api_key)

In [47]:
index = pc.Index(index_name)

In [48]:
from langchain.vectorstores import Pinecone

# Initialize the Pinecone vector store
vector_store = Pinecone(
    index=index,  # Your Pinecone index object
    embedding=gemini_embeddings,  # Embedding function
    text_key="text"  # Metadata key where document text is stored
)

In [59]:
type(vector_store)

langchain_community.vectorstores.pinecone.Pinecone

In [60]:
from langchain.chains import RetrievalQA
def aq(query):
    values = {"model": "gemini-1.5-flash-latest"}  # or another supported model
    llm = ChatGoogleGenerativeAI(**values)
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs ={'k' : 3})
    chain = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever=retriever)
    ans = chain.run(query)
    return ans

In [65]:
q = "What is the context of document ?"


In [66]:
display(Markdown(aq(q)))

The context of the document is the **U.S. Constitution, specifically Article III, Section 2, Clause 3**. This section deals with the jurisdiction of the Supreme Court. 

The portion you provided describes the types of cases the Supreme Court has original jurisdiction over, meaning cases that can be filed directly with the Supreme Court. 


In [63]:
import time
i = 1
print('Write quit or exit to leave terminal')
while True:
    q = input(f'Question #{i}: ')
    i= i + 1
    if q.lower() in ['quit', 'exit']:
        print('Leaving terminal...')
        time.sleep(2)
        break
    
    answer = aq(q)  # Get the answer from the function
    display(Markdown(answer))  # Display the answer as Markdown  # Print the answer as plain text
    print(f'\n{"-"*50}\n')

Write quit or exit to leave terminal
Question #1: explanation Amendment 


The provided text is a fragment of a proposed amendment to the United States Constitution.  It doesn't provide the full context of what the amendment would actually change. 

Here's what we can glean from the text:

* **Repeal:** The amendment starts by repealing a previous section (Section 2) of the Constitution. 
* **Ratification Process:** The amendment outlines the ratification process, requiring approval by the legislatures of the states within seven years of its submission. 
* **Temporary Appointments:** The amendment discusses temporary appointments until vacancies are filled by election, potentially related to the Senate.
* **Election and Term of Senators:** The amendment clarifies that it won't affect the election or term of any Senator already chosen before the amendment becomes part of the Constitution.

**To fully understand the amendment, we would need to know:**

* **What was the content of Section 2 that is being repealed?**
* **What specific changes to the process of filling vacancies in the Senate are being proposed?**

Without that information, it's impossible to provide a full explanation of the amendment. 



--------------------------------------------------

Question #2: quit
Leaving terminal...


In [67]:
def load_from_wikipedia(query,lang = 'en', load_max_docs=2):
    from langchain.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=2)
    data = loader.load()
    return data

In [97]:
data = load_from_wikipedia("Kolkata Case")
print(data[0].page_content)

On 9 August 2024, Moumita Debnath, a trainee doctor at R. G. Kar Medical College in Kolkata, West Bengal, India, was raped and murdered in a college building. Her body was found in a seminar room on campus. The incident has amplified debate about the safety of women and doctors in India, and has sparked significant outrage, nationwide and international protests, and demands for a thorough investigation.


== Incident ==

On the night of 8 August 2024, Debnath had dinner with her colleagues and retired in a seminar hall after a 36-hour shift in the hospital. At about 9:30 IST the following morning, her body was discovered in the seminar hall in a semi-nude state with her eyes, mouth, and genitals bleeding. She was later declared dead. College authorities initially informed her family that she had committed suicide. 
An autopsy was conducted on 9 August and, after its completion, an unnatural death case was registered.


== Investigation ==


=== Autopsy report ===
An autopsy revealed th

In [99]:
data

[Document(metadata={'title': '2024 Kolkata rape and murder incident', 'summary': 'On 9 August 2024, Moumita Debnath, a trainee doctor at R. G. Kar Medical College in Kolkata, West Bengal, India, was raped and murdered in a college building. Her body was found in a seminar room on campus. The incident has amplified debate about the safety of women and doctors in India, and has sparked significant outrage, nationwide and international protests, and demands for a thorough investigation.', 'source': 'https://en.wikipedia.org/wiki/2024_Kolkata_rape_and_murder_incident'}, page_content='On 9 August 2024, Moumita Debnath, a trainee doctor at R. G. Kar Medical College in Kolkata, West Bengal, India, was raped and murdered in a college building. Her body was found in a seminar room on campus. The incident has amplified debate about the safety of women and doctors in India, and has sparked significant outrage, nationwide and international protests, and demands for a thorough investigation.\n\n\n=

In [100]:
type(data)

list

In [103]:
delete_pinecone_index()

Deleting all indexes..
done


In [101]:
chunks = text_splitter.split_documents(data)

In [102]:
chunks[0].page_content

'On 9 August 2024, Moumita Debnath, a trainee doctor at R. G. Kar Medical College in Kolkata, West Bengal, India, was raped and murdered in a college building. Her body was found in a seminar room on campus. The incident has amplified debate about the'

In [104]:
index_name = "gemini"
vector_store = insert_embeddings(index_name)

Creating index gemini and embeddings.

In [105]:
index = pc.Index(index_name)

In [110]:
from langchain.vectorstores import Pinecone

# Initialize the Pinecone vector store
vector_store = Pinecone(
    index=index,  # Your Pinecone index object
    embedding=gemini_embeddings,  # Embedding function
    text_key="text"  # Metadata key where document text is stored
)

In [106]:
type(vector_store)

NoneType

In [107]:
from langchain.chains import RetrievalQA
def aq(query):
    values = {"model": "gemini-1.5-flash-latest"}  # or another supported model
    llm = ChatGoogleGenerativeAI(**values)
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs ={'k' : 3})
    chain = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever=retriever)
    ans = chain.run(query)
    return ans

In [116]:
q = 'what happened at nigh of 25th'

In [117]:
display(Markdown(aq(q)))

The CBI raided the house of Sandip Ghosh and Sanjay Vashisth on the night of 25 August. 
